In [91]:
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
import warnings
warnings.filterwarnings(action='ignore')
#import descartes
import folium

In [5]:
%cd E:\yujin\INHA\data

E:\yujin\INHA\data


# 열 순서

category, 좌표값, 인구분포,면적, 주차대수, 행정동 정보,행정동 별 전기차 개수

# 주차장 전처리

#### 02.광양시_주차장_공간정보 + 26.광양시_도시계획(교통시설)

## 1) 02.광양시_주차장_공간정보 --> parkingLot

In [122]:
gyboundary = gpd.read_file('20.광양시_행정경계(읍면동).geojson')
gyboundary

,ADM_DR_CD,ADM_DR_NM,geometry
0,3606031,봉강면,"MULTIPOLYGON (((127.58037 35.10902, 127.58052 ..."
1,3606032,옥룡면,"MULTIPOLYGON (((127.59872 35.12006, 127.59919 ..."
2,3606033,옥곡면,"MULTIPOLYGON (((127.64878 35.03547, 127.64902 ..."
3,3606034,진상면,"MULTIPOLYGON (((127.65639 35.11680, 127.65698 ..."
4,3606035,진월면,"MULTIPOLYGON (((127.75715 35.05502, 127.75716 ..."
5,3606036,다압면,"MULTIPOLYGON (((127.64117 35.16850, 127.64140 ..."
6,3606051,골약동,"MULTIPOLYGON (((127.68056 34.92848, 127.68105 ..."
7,3606053,중마동,"MULTIPOLYGON (((127.70376 34.96198, 127.70542 ..."
8,3606054,광영동,"MULTIPOLYGON (((127.71676 34.96939, 127.71691 ..."
9,3606055,태인동,"MULTIPOLYGON (((127.76162 34.95988, 127.76163 ..."


In [132]:
parkingLot = pd.read_csv('02.광양시_주차장_공간정보.csv')

parkingLot['lon'] = parkingLot['lon'].astype(float)
parkingLot['lat'] = parkingLot['lat'].astype(float)
parkingLot['geometry'] = parkingLot.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
parkingLot = gpd.GeoDataFrame(parkingLot, geometry='geometry')
parkingLot = parkingLot[['면적','주차면수','geometry']]
parkingLot.rename(columns={"면적":"area"}, inplace=True)

pllist = []
for i in range(len(parkingLot)):
    for j in range(len(gyboundary)):
        if parkingLot.geometry[i].within(gyboundary.geometry[j]) == True:
            pllist.append(gyboundary.ADM_DR_NM[j])

parkingLot['administration'] = pllist
parkingLot

,area,주차면수,geometry,administration
0,4189.0,161.0,POINT (127.58688 34.97032),광양읍
1,2968.0,125.0,POINT (127.69543 34.93625),중마동
2,2836.0,89.0,POINT (127.58459 34.97564),광양읍
3,1980.0,52.0,POINT (127.58068 34.97013),광양읍
4,1970.0,64.0,POINT (127.58071 34.97252),광양읍
...,...,...,...,...
230,264.0,10.0,POINT (127.60482 35.07921),옥룡면
231,1212.0,60.0,POINT (127.61342 35.07067),옥룡면
232,2381.0,96.0,POINT (127.61177 35.00276),옥룡면
233,2090.0,66.0,POINT (127.67472 35.09239),진상면


## 2) 26.광양시_도시계획(교통시설) --> traffic_facility_parkingLot

In [203]:
traffic_facility = gpd.read_file('26.광양시_도시계획(교통시설).geojson')

traffic_facility_parkingLot = traffic_facility[traffic_facility['DGM_NM'].str.contains('주차')]  #교통시설 중 '주차'가 들어간 것만 추출
traffic_facility_parkingLot.rename(columns={"DGM_AR":"area"}, inplace=True)  #면적 의미하는 DGM_AR변수명을 area로 변경
traffic_facility_parkingLot['center_x']=traffic_facility_parkingLot['geometry'].centroid.x  #multipoligon형태여서 중심값 추출
traffic_facility_parkingLot['center_y']=traffic_facility_parkingLot['geometry'].centroid.y
traffic_facility_parkingLot['geometry'] = traffic_facility_parkingLot.apply(lambda row : Point([row['center_x'], row['center_y']]), axis=1)
traffic_facility_parkingLot.drop(['center_x','center_y', 'PRESENT_SN','DGM_LT','DGM_NM'], axis=1, inplace=True)  #필요없는 열들 제거

traffic_facility_parkingLot.reset_index(drop=True, inplace=True)
traffic_facility_parkingLot

,area,geometry
0,344,POINT (127.58425 34.97704)
1,483,POINT (127.58390 34.97837)
2,1612,POINT (127.75138 34.94576)
3,305,POINT (127.70270 34.98894)
4,4608,POINT (127.68964 34.93745)
...,...,...
203,805,POINT (127.58994 34.97387)
204,1244,POINT (127.66659 34.93274)
205,2657,POINT (127.58461 34.97563)
206,1313,POINT (127.69850 34.92671)


## 2-1) 도시계획_주차장 208개 중 행정구역에 속해 있지 않은 4개의 주차장 삭제

In [204]:
#읍면동 넣으려고 했는데 계속 에러나길래 이유가 뭔가 했더니 4개의 주차장이 광양시 행정구역에 안속해 있었음 ㅠㅠ

lst = []
for i in range(len(traffic_facility_parkingLot)):
    for j in range(len(gyboundary)):
        if traffic_facility_parkingLot.geometry[i].within(gyboundary.geometry[j]) == True:
            lst.append(gyboundary.ADM_DR_NM[j])

print("행정구역에 속한 주차장 개수 :", len(lst))

행정구역에 속한 주차장 개수 : 204


In [205]:
lst2 = []
for i in range(len(traffic_facility_parkingLot)):
    for j in range(len(gyboundary)):
        if traffic_facility_parkingLot.geometry[i].within(gyboundary.geometry[j]) == True:
            lst2.append(i)
            
print("행정구역에 속하지 않은 주차장 index :")
for i in range(len(traffic_facility_parkingLot)):
    if i not in lst2:
        print(i)

행정구역에 속하지 않은 주차장 index :
153
154
155
156


In [206]:
#153, 154, 155, 156에 해당하는 주차장 삭제

traffic_facility_parkingLot = traffic_facility_parkingLot.drop([traffic_facility_parkingLot.index[153],traffic_facility_parkingLot.index[154],
                                                               traffic_facility_parkingLot.index[155],traffic_facility_parkingLot.index[156]])
traffic_facility_parkingLot.reset_index(drop=True,inplace=True)
traffic_facility_parkingLot

,area,geometry
0,344,POINT (127.58425 34.97704)
1,483,POINT (127.58390 34.97837)
2,1612,POINT (127.75138 34.94576)
3,305,POINT (127.70270 34.98894)
4,4608,POINT (127.68964 34.93745)
...,...,...
199,805,POINT (127.58994 34.97387)
200,1244,POINT (127.66659 34.93274)
201,2657,POINT (127.58461 34.97563)
202,1313,POINT (127.69850 34.92671)


In [207]:
# 다시 읍면동 넣기
tlist = []
for i in range(len(traffic_facility_parkingLot)):
    for j in range(len(gyboundary)):
        if traffic_facility_parkingLot.geometry[i].within(gyboundary.geometry[j]) == True:
            tlist.append(gyboundary.ADM_DR_NM[j])

traffic_facility_parkingLot['administration'] = tlist
traffic_facility_parkingLot

,area,geometry,administration
0,344,POINT (127.58425 34.97704),광양읍
1,483,POINT (127.58390 34.97837),광양읍
2,1612,POINT (127.75138 34.94576),태인동
3,305,POINT (127.70270 34.98894),옥곡면
4,4608,POINT (127.68964 34.93745),중마동
...,...,...,...
199,805,POINT (127.58994 34.97387),광양읍
200,1244,POINT (127.66659 34.93274),골약동
201,2657,POINT (127.58461 34.97563),광양읍
202,1313,POINT (127.69850 34.92671),중마동


## 2-2) 행정구역에 속하지 않은 주차장 위치 확인

In [101]:
설치현황 = pd.read_csv('01.광양시_충전기설치현황.csv')
lat = 설치현황['lat'].mean()
lon = 설치현황['lon'].mean()


m = folium.Map([lat,lon], zoom_start=11)
folium.GeoJson(gyboundary).add_to(m)

for i in traffic_facility_parkingLot.index:
    sub_lat = traffic_facility_parkingLot.loc[i,"center_y"]
    sub_lon = traffic_facility_parkingLot.loc[i,'center_x']
    title = i
    folium.Marker([sub_lat, sub_lon], tooltip=title).add_to(m)
m

## 3) 두 테이블 합치기 (parkingLot + traffic_facility_parkingLot)

In [212]:
new_parkingLot = pd.concat([parkingLot, traffic_facility_parkingLot], ignore_index=True)
new_parkingLot

,area,주차면수,geometry,administration
0,4189.0,161.0,POINT (127.58688 34.97032),광양읍
1,2968.0,125.0,POINT (127.69543 34.93625),중마동
2,2836.0,89.0,POINT (127.58459 34.97564),광양읍
3,1980.0,52.0,POINT (127.58068 34.97013),광양읍
4,1970.0,64.0,POINT (127.58071 34.97252),광양읍
...,...,...,...,...
434,805.0,NaN,POINT (127.58994 34.97387),광양읍
435,1244.0,NaN,POINT (127.66659 34.93274),골약동
436,2657.0,NaN,POINT (127.58461 34.97563),광양읍
437,1313.0,NaN,POINT (127.69850 34.92671),중마동


# 야영장 전처리

## 1) 경위도 --> geometry,   administration 추가

In [222]:
야영장

,명칭,주소,소유주체(공공/민간),lon,lat
0,진틀펜션야영장,광양시 옥룡면 신재로 1678-54,민간,127.601787,35.096566
1,느랭이골자연휴양림,광양시 다압면 토끼재길 119-32,민간,127.710849,35.060546
2,백운산수련관야영장,광양시 옥룡면 약수제단길 154,민간,127.631555,35.069553
3,봉강하조오토캠핑장,광양시 봉강면 성불로 1097,민간,127.562487,35.076193
4,백운산자연휴양림,광양시 옥룡면 백계로 337,공공,127.591184,35.054216
5,백운제오토캠핑장,광양시 봉강면 봉당리 178,공공,127.581644,35.005226


In [218]:
야영장 = pd.read_csv("05.광양시_대중집합시설_야영장.csv")
campsite = 야영장[['소유주체(공공/민간)','lon', 'lat']]
campsite['소유주체(공공/민간)'] = np.where(campsite['소유주체(공공/민간)']=='민간',0,1) #민간 = 0, 공공 = 1
campsite

,소유주체(공공/민간),lon,lat
0,0,127.601787,35.096566
1,0,127.710849,35.060546
2,0,127.631555,35.069553
3,0,127.562487,35.076193
4,1,127.591184,35.054216
5,1,127.581644,35.005226


In [220]:
campsite['geometry'] = campsite.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
campsite.drop(['lon','lat'], axis=1, inplace=True)
campsite

,소유주체(공공/민간),geometry
0,0,POINT (127.601787 35.096566)
1,0,POINT (127.710849 35.060546)
2,0,POINT (127.631555 35.06955300000001)
3,0,POINT (127.562487 35.076193)
4,1,POINT (127.591184 35.054216)
5,1,POINT (127.581644 35.005226)


In [221]:
camp_list = []
for i in range(len(campsite)):
    for j in range(len(gyboundary)):
        if campsite.geometry[i].within(gyboundary.geometry[j]) == True:
            camp_list.append(gyboundary.ADM_DR_NM[j])

campsite['administration'] = camp_list
campsite

,소유주체(공공/민간),geometry,administration
0,0,POINT (127.601787 35.096566),옥룡면
1,0,POINT (127.710849 35.060546),다압면
2,0,POINT (127.631555 35.06955300000001),옥룡면
3,0,POINT (127.562487 35.076193),봉강면
4,1,POINT (127.591184 35.054216),옥룡면
5,1,POINT (127.581644 35.005226),봉강면


# 공공문화 체육시설

In [223]:
공공시설 = gpd.read_file('25.광양시_도시계획(공공문화체육시설).geojson')
공공시설

,PRESENT_SN,DGM_NM,DGM_AR,DGM_LT,geometry
0,46230UQ155PS201202010089,청사(국가),22707,591,"MULTIPOLYGON (((127.69671 34.93974, 127.69665 ..."
1,46230UQ155PS201202010085,중학교,17094,516,"MULTIPOLYGON (((127.69302 34.93371, 127.69219 ..."
2,46230UQ155PS201202010084,고등학교,19265,537,"MULTIPOLYGON (((127.69125 34.94687, 127.69123 ..."
3,46230UQ155PS201202010083,초등학교,12998,479,"MULTIPOLYGON (((127.69033 34.95532, 127.69029 ..."
4,46230UQ155PS201202010082,초등학교,20596,573,"MULTIPOLYGON (((127.68677 34.94771, 127.68671 ..."
...,...,...,...,...,...
93,46230UQ155PS201809120065,도서관,810,117,"MULTIPOLYGON (((127.60749 34.97396, 127.60731 ..."
94,46230UQ155PS201202010032,기타 체육시설,20383,586,"MULTIPOLYGON (((127.69846 34.98736, 127.69847 ..."
95,46230UQ155PS201202010098,초등학교,37090,808,"MULTIPOLYGON (((127.72189 34.93462, 127.72188 ..."
96,46230UQ155PS201202010097,중학교,18208,536,"MULTIPOLYGON (((127.71951 34.95927, 127.71951 ..."
